In [1]:
import pandas as pd
import numpy as np
import warnings
from functions import metrics
import csv
from sklearn.exceptions import ConvergenceWarning
from functions.formatting import get_subgroup_str
from itertools import product, combinations
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [27]:
omit_demographics = True 
use_gerryfair = False 

In [28]:
df = pd.read_csv('./data/preprocessed.csv')
df.drop(['umich_user_id', 'Unnamed: 0'], axis=1, inplace=True)

In [29]:
X = df.drop('completed', axis=1)
y = df['completed']

#### Calculate false positive rates for the given subgroup

This function takes in the subgroup to test for as a list of tuples. We use logistic regression and 10-fold cross validation to calculate the average FPR and AUC for the given subgroup. 

Returns a tuple containing FPR average, FPR standard deviation, AUC average, AUC standard deviation, and number of students in a given subgroup

#### Compute all subgroups

This function generates all possible subgroups given the list of protected columns

In [30]:
demo_groups = [['US', 'intl'], ['bachelor_obtained', 'no_bachelor_obtained', 'education_na'], ['white', 'black', 'asian', 'latinx', 'race_others', 'race_na'], ['male', 'female', 'gender_na', 'gender_other']]

In [31]:
def compute_combos():
    ret = []
    for i in range(1,len(demo_groups)+1):
        g = list(combinations(demo_groups, i))
        for demos in g:
            ret += list(product(*demos))

    return ret

In [32]:
combos = compute_combos()

In [33]:
combos.sort(key=len)
# combos
combos.insert(0, ())

In [34]:
combos

[(),
 ('US',),
 ('intl',),
 ('bachelor_obtained',),
 ('no_bachelor_obtained',),
 ('education_na',),
 ('white',),
 ('black',),
 ('asian',),
 ('latinx',),
 ('race_others',),
 ('race_na',),
 ('male',),
 ('female',),
 ('gender_na',),
 ('gender_other',),
 ('US', 'bachelor_obtained'),
 ('US', 'no_bachelor_obtained'),
 ('US', 'education_na'),
 ('intl', 'bachelor_obtained'),
 ('intl', 'no_bachelor_obtained'),
 ('intl', 'education_na'),
 ('US', 'white'),
 ('US', 'black'),
 ('US', 'asian'),
 ('US', 'latinx'),
 ('US', 'race_others'),
 ('US', 'race_na'),
 ('intl', 'white'),
 ('intl', 'black'),
 ('intl', 'asian'),
 ('intl', 'latinx'),
 ('intl', 'race_others'),
 ('intl', 'race_na'),
 ('US', 'male'),
 ('US', 'female'),
 ('US', 'gender_na'),
 ('US', 'gender_other'),
 ('intl', 'male'),
 ('intl', 'female'),
 ('intl', 'gender_na'),
 ('intl', 'gender_other'),
 ('bachelor_obtained', 'white'),
 ('bachelor_obtained', 'black'),
 ('bachelor_obtained', 'asian'),
 ('bachelor_obtained', 'latinx'),
 ('bachelor_obt

In [35]:
demographics = ['US', 'intl', 'bachelor_obtained', 'no_bachelor_obtained', 'education_na', 'white', 'black', 'asian', 'latinx', 'race_others', 'race_na', 'male', 'female', 'gender_na', 'gender_other']
protected=['US', 'intl', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female']

In [36]:
res = metrics.calc_metrics(X, y, combos, omit_demographics=False, demographics=demographics, protected=protected, is_gerryfair=False, iters=3)

/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_di

In [ ]:
# res

Generate a list of all subgroup data and write it to a csv

In [ ]:
csv_file = f"./data/MTC508_subgroup_data_with_demographics.csv"
# csv_file = "test.csv"

# Write the data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    fieldnames = [
        'subgroup', 'n', 'auc_avg', 'auc_std', 'fpr_avg', 'fpr_std', 'rmse_avg', 'rmse_std']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()  # Write the header row
    for row in res:
        writer.writerow(row)

In [ ]:
# data = {}

# data['overall'] = len(X)

# for c in combos[1:]:
#     masks = [X[name] == value for name, value in c]
#     final_mask = pd.concat(masks, axis=1).all(axis=1)
#     filter_X = X[final_mask]
#     data[get_subgroup_str(c)] = len(filter_X)




In [ ]:
# df = pd.DataFrame(list(data.items()), columns=['Key', 'Count'])

# # Specify the CSV file path
# csv_file_path = 'data/MTC508_data_counts.csv'

# # Write the DataFrame to a CSV file
# df.to_csv(csv_file_path, index=False)

In [ ]:
subgroup_data = pd.read_csv('./data/MTC508_subgroup_data_without_demographics.csv')
fpr_data = {}
# loop through each subgroup
# for each subgroup, get the corresponding data
for index, row in subgroup_data.iterrows():
    # print(f"Index: {index}, fpr: {row['fpr_avg']}")
    fpr_data[row['subgroup']] = row['fpr_avg']



In [ ]:
def get_data_iters(start, stop, step, attrs):
    ret_val = []
    data = []
    for i in range(start+1,stop+1,step):
        data.append(metrics.calc_metrics(X, y, combos, omit_demographics=True, demographics=demographics, protected=protected, is_gerryfair=True, iters=i))
    
    for a in attrs:
        fprs = []
        for res in data:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break

        iterations = np.array(list(range(start, stop, step)))

        other_algorithm_fpr = np.linspace(fpr_data[a], fpr_data[a], iterations.size)

        ret_val.append((iterations, fprs, other_algorithm_fpr))
    
    return ret_val
        


In [2]:
attrs = ['Overall', 'white', 'black', 'female', 'black, female']

In [ ]:
val = get_data_iters(0, 1001, 20, attrs)

In [24]:
val

[(array([   0,   20,   40,   60,   80,  100,  120,  140,  160,  180,  200,
          220,  240,  260,  280,  300,  320,  340,  360,  380,  400,  420,
          440,  460,  480,  500,  520,  540,  560,  580,  600,  620,  640,
          660,  680,  700,  720,  740,  760,  780,  800,  820,  840,  860,
          880,  900,  920,  940,  960,  980, 1000]),
  [0.164,
   0.119,
   0.082,
   0.08,
   0.08,
   0.077,
   0.071,
   0.073,
   0.07,
   0.07,
   0.071,
   0.073,
   0.072,
   0.072,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.07,
   0.07,
   0.07,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.07,
   0.07,
   0.072,
   0.072,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.068,
   0.068,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07],
  array([0.169, 0.169, 0.169, 0.169, 0.169, 0.169, 0.169, 0.169, 0.169,
         0.169, 0.169, 0.169, 0.169, 0.169, 0.169, 0.169, 0.169, 0.169,
         0.169,

In [ ]:
for i, g in enumerate(attrs):
    iterations, fprs, other_algorithm_fpr = val[i]

    df = pd.DataFrame({'x_axis': iterations, 'y_axis': fprs, 'other': other_algorithm_fpr})
    df.to_csv('./data/graph_iters_fpr_0_1001_20' + g + '.csv')

    

In [51]:
val = []
for i, g in enumerate(attrs):
    data = pd.read_csv('./data/graph_iters_fpr_0_1001_20' + g + '.csv')
    line = [list(data['x_axis']), list(data['y_axis']), list(data['other'])]

    val.append(line)
    

In [52]:
val

[[[0,
   20,
   40,
   60,
   80,
   100,
   120,
   140,
   160,
   180,
   200,
   220,
   240,
   260,
   280,
   300,
   320,
   340,
   360,
   380,
   400,
   420,
   440,
   460,
   480,
   500,
   520,
   540,
   560,
   580,
   600,
   620,
   640,
   660,
   680,
   700,
   720,
   740,
   760,
   780,
   800,
   820,
   840,
   860,
   880,
   900,
   920,
   940,
   960,
   980,
   1000],
  [0.164,
   0.119,
   0.082,
   0.08,
   0.08,
   0.077,
   0.071,
   0.073,
   0.07,
   0.07,
   0.071,
   0.073,
   0.072,
   0.072,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.07,
   0.07,
   0.07,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.068,
   0.07,
   0.07,
   0.072,
   0.072,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.068,
   0.068,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07,
   0.07],
  [0.169,
   0.169,
   0.169,
   0.169,
   0.169,
   0.169,
   0.169,
   0.169,
   0.169,
   0.169,
   0.169

In [53]:
def build_graph(x_axis, y_axis, other_algorithm_fpr, x_axis_label, y_axis_label, a):
        plt.plot(x_axis, y_axis, linestyle='-', label='GerryFair')

        plt.plot(x_axis, other_algorithm_fpr, linestyle='-', label='LR Without Gerryfair')
        plt.ylim(0, 1)

        plt.xlabel(x_axis_label)
        plt.ylabel(y_axis_label)
        plt.title(f'Comparison of Algorithms - {y_axis_label} vs {x_axis_label} For ' + a.capitalize())

        plt.legend()

        plt.show()

In [12]:
attributes = ["Overall", "White", "Black", "Female", "Black + Female"]

for i, attr in enumerate(attributes):
    build_graph(val[i][0], val[i][1], val[i][2], 'Iterations', 'FPR', attr)

In [54]:
import matplotlib.pyplot as plt

def build_combined_graph(val, attributes):
    # Set distinct colors for each attribute
    colors = ['b', 'g', 'r', 'c', 'm']  # You can add more colors if needed

    # Create a single plot
    plt.figure(figsize=(12, 6))  # Adjust the figure size as needed

    for i, attr in enumerate(attributes):
        x_axis, y_axis, other_algorithm_fpr = val[i]
        plt.plot(x_axis, y_axis, linestyle='-', label=f'GerryFair - {attr.capitalize()}', color=colors[i])
        plt.plot(x_axis, other_algorithm_fpr, linestyle='-', label=f'LR Without Gerryfair - {attr.capitalize()}', color=colors[i], alpha=0.5)

    # Set the y-axis range to 0 to 1
    plt.ylim(0, 1)

    plt.xlabel('Iterations')
    plt.ylabel('FPR')
    plt.title('Comparison of Algorithms - FPR vs Iterations')

    # Move the legend outside the axes
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    # Automatically adjust subplot parameters to give specified padding
    plt.tight_layout()

    plt.show()

attributes = ["Overall", "White", "Black", "Female", "Black + Female"]

build_combined_graph(val, attributes)


In [ ]:
def build_graph_gamma(iters, gammas, attrs):
    results = []
    for g in gammas:
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=iters+1, gamma=g))
    
    data = results
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        gammas = np.array(gammas)

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], gammas.size) 

        plt.plot(gammas, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(gammas, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Gamma')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Gamma For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_gamma(5, [.002, .005, .01, .02, .05, .1], ['Overall', 'black', 'female', 'black, female'])

In [13]:
len(data)

51

In [ ]:
def gen_data_attrs(iters, attrs, sets):
    results = []
    for i in range(len(sets)):
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=demographics[:i+1], is_gerryfair=True, iters=iters+1, gamma=g))
    
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        num_attrs = np.array(list(range(1,len(demographics) + 1)))

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], len(demographics)) 

        plt.plot(num_attrs, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(num_attrs, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Number of Attributes')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Number of Attributes For ' + a.capitalize())

        plt.legend()

        plt.show()

In [48]:
gerryfair_groups = {
    'all': protected,
    'location': demo_groups[0], 
    'education': demo_groups[1][:-1], 
    'race': demo_groups[2][:-2], 
    'gender': demo_groups[3][:-2],
    'race+gender': demo_groups[2][:-2] + demo_groups[3][:-2]
    }

In [49]:
def gen_data_categories(iters, attrs):
    for g in attrs:
        res = metrics.calc_metrics(X, y, combos, omit_demographics=False, demographics=demographics, protected=attrs[g], is_gerryfair=True, iters=iters)
        csv_file = f"./data/graph_attrs_{str(iters)}_{g}.csv"

        # Write the data to a CSV file
        with open(csv_file, 'w', newline='') as csvfile:
            fieldnames = [
                'subgroup', 'n', 'auc_avg', 'auc_std', 'fpr_avg', 'fpr_std', 'rmse_avg', 'rmse_std']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()  # Write the header row
            for row in res:
                writer.writerow(row)

In [50]:
gen_data_categories(200, gerryfair_groups)

iteration: 1
most accurate classifier accuracy: 0.21988950276243094, most acc-class unfairness: 0.027888541916886864, most acc-class size 0.3425414364640884
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
barrier
iteration: 29
barrier
iteration: 30
iteration: 31
iteration: 32
iteration: 33
iteration: 34
iteration: 35
iteration: 36
iteration: 37
iteration: 38
iteration: 39
iteration: 40
iteration: 41
iteration: 42
iteration: 43
iteration: 44
iteration: 45
iteration: 46
iteration: 47
iteration: 48
barrier
iteration: 49
barrier
iteration: 50
barrier
iteration: 51
barrier
iteration: 52
barrier
iteration: 53
barrier
iteration: 54
barrier
iteration: 55
barrier
iteration: 56
b

/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rco

iteration: 1
most accurate classifier accuracy: 0.21988950276243094, most acc-class unfairness: 0.006653855392745617, most acc-class size 0.07734806629834255
barrier
iteration: 2
barrier
iteration: 3
barrier
iteration: 4
barrier
iteration: 5
barrier
iteration: 6
barrier
iteration: 7
barrier
iteration: 8
barrier
iteration: 9
barrier
iteration: 10
barrier
iteration: 11
barrier
iteration: 12
barrier
iteration: 13
barrier
iteration: 14
barrier
iteration: 15
barrier
iteration: 16
barrier
iteration: 17
barrier
iteration: 18
barrier
iteration: 19
barrier
iteration: 20
barrier
iteration: 21
barrier
iteration: 22
barrier
iteration: 23
barrier
iteration: 24
barrier
iteration: 25
barrier
iteration: 26
barrier
iteration: 27
barrier
iteration: 28
barrier
iteration: 29
barrier
iteration: 30
barrier
iteration: 31
barrier
iteration: 32
barrier
iteration: 33
barrier
iteration: 34
barrier
iteration: 35
barrier
iteration: 36
barrier
iteration: 37
barrier
iteration: 38
barrier
iteration: 39
barrier
iterat

/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rco

iteration: 1
most accurate classifier accuracy: 0.21988950276243094, most acc-class unfairness: 0.006106173432620707, most acc-class size 0.041988950276243095
barrier
iteration: 2
barrier
iteration: 3
barrier
iteration: 4
barrier
iteration: 5
barrier
iteration: 6
barrier
iteration: 7
barrier
iteration: 8
barrier
iteration: 9
barrier
iteration: 10
barrier
iteration: 11
barrier
iteration: 12
barrier
iteration: 13
barrier
iteration: 14
barrier
iteration: 15
barrier
iteration: 16
barrier
iteration: 17
barrier
iteration: 18
barrier
iteration: 19
barrier
iteration: 20
barrier
iteration: 21
barrier
iteration: 22
barrier
iteration: 23
barrier
iteration: 24
barrier
iteration: 25
barrier
iteration: 26
barrier
iteration: 27
barrier
iteration: 28
barrier
iteration: 29
barrier
iteration: 30
barrier
iteration: 31
barrier
iteration: 32
barrier
iteration: 33
barrier
iteration: 34
barrier
iteration: 35
barrier
iteration: 36
barrier
iteration: 37
barrier
iteration: 38
barrier
iteration: 39
barrier
itera

...

/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rco

iteration: 1
most accurate classifier accuracy: 0.21988950276243094, most acc-class unfairness: 0.014926735527263993, most acc-class size 0.1834254143646409
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
barrier
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
barrier
iteration: 20
barrier
iteration: 21
barrier
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
iteration: 29
iteration: 30
iteration: 31
iteration: 32
iteration: 33
iteration: 34
barrier
iteration: 35
barrier
iteration: 36
barrier
iteration: 37
barrier
iteration: 38
barrier
iteration: 39
barrier
iteration: 40
barrier
iteration: 41
barrier
iteration: 42
barrier
iteration: 43
barrier
iteration: 44
barrier
iteration: 45
barrier
iteration: 46
barrier
iteration: 47
barrier
iteration: 48
barrier
iteration: 49
barrier
iteration: 50
itera

/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rco

iteration: 1
most accurate classifier accuracy: 0.21988950276243094, most acc-class unfairness: 0.02693970694210906, most acc-class size 0.3370165745856354
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
barrier
iteration: 23
barrier
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
iteration: 29
iteration: 30
iteration: 31
iteration: 32
iteration: 33
iteration: 34
iteration: 35
iteration: 36
iteration: 37
iteration: 38
iteration: 39
iteration: 40
iteration: 41
barrier
iteration: 42
barrier
iteration: 43
barrier
iteration: 44
barrier
iteration: 45
barrier
iteration: 46
barrier
iteration: 47
barrier
iteration: 48
barrier
iteration: 49
barrier
iteration: 50
barrier
iteration: 51
barrier
iteration: 52
iteration: 53
iteration: 54
iterat

/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rco

In [18]:
demographics

['US',
 'intl',
 'bachelor_obtained',
 'no_bachelor_obtained',
 'education_na',
 'white',
 'black',
 'asian',
 'latinx',
 'race_others',
 'race_na',
 'male',
 'female',
 'gender_na',
 'gender_other']

In [44]:
demo_groups[2][-2]

'race_others'

In [23]:
protected

['US',
 'intl',
 'bachelor_obtained',
 'white',
 'black',
 'asian',
 'latinx',
 'male',
 'female']